In [1]:
import sys
import os
import polars as pl
import numpy as np
# Add the project root directory to PYTHONPATH
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
sys.path.insert(0, project_root)

# Import necessary modules
import groundinsight as gi
from groundinsight.models.core_models import BusType, BranchType




In [2]:

#Test the calculation formula of the impedance
# Create a network
net = gi.create_network(name="MyTestNetwork", frequencies=[0, 50, 250, 350, 450, 550])
net.description = "That's my first test network"

bus_type = BusType(
    name="BusTypeFormulaTest",
    description="Example bus type with parameters",
    system_type="Grounded",
    voltage_level=230.0,
    impedance_formula="rho * 0 + 1 + I * f * 1/50",
)

bus_type_uw = BusType(
    name="BusTypeFormulaTestUW",
    description="Example bus type with parameters",
    system_type="Grounded",
    voltage_level=230.0,
    impedance_formula="rho * 0 + 0.1 + I * f * 1/50",
)

branch_type = BranchType(
    name="TestBranchType",
    description="A test branch type",
    grounding_conductor=True,
    self_impedance_formula="(rho * 0 + 0.25 + j * f * 0.012)*l",
    mutual_impedance_formula="(rho * 0 + 0.0 + j * f * 0.010)*l"
)

branch_ohl = BranchType(
    name="OHLine",
    description="An overhead line",
    grounding_conductor=False,
    self_impedance_formula="NaN",
    mutual_impedance_formula="NaN"
)

number_buses = 30
#create buses with a for loop
for i in range(1, number_buses):
    gi.create_bus(name=f"bus{i}", type=bus_type, network=net, specific_earth_resistance=100.0)


#create branch 
#defining a line length of each branch
line_length = 1

#create bracnches with a for loop
for i in range(1, number_buses-1):
    gi.create_branch(name=f"branch{i}", type=branch_type, from_bus=f"bus{i}", to_bus=f"bus{i+1}", length=line_length, specific_earth_resistance=100.0, network=net)


#net.add_bus(bus2, overwrite=True)

#create a faults
fault_scaling = {0:1, 50: 1.0, 250: 1}
faultnames = []
for i in range(1, number_buses):
    gi.create_fault(name=f"fault{i}", bus=f"bus{i}", description=f"A fault at bus {i}", scalings=fault_scaling, network=net)
    faultnames.append(f"fault{i}")


#add a source at bus1
source = gi.create_source(name="source1", bus="bus1", values={0:10, 50:60, 250:60, 350:60, 450:60, 550:60}, network=net)
#source2 = gi.create_source(name="source2", bus="bus6", values={0:0, 50:30, 250:30, 350:30}, network=net)

#define the paths of the network
gi.create_paths(network=net)

#for loop over each fault
for i in range(number_buses-1):
    # Run fault calculations
    gi.run_fault(net, fault_name=f"fault{i+1}")





In [3]:
#try the json saving
path = "network.json"
gi.save_network_to_json(net, path)

reconvert_net = gi.load_network_from_json(path)
print(net.faults)
print(reconvert_net.faults)
#define the paths of the network
gi.create_paths(network=reconvert_net)

#for loop over each fault
for i in range(9):
    # Run fault calculations
    gi.run_fault(reconvert_net, fault_name=f"fault{i+1}")

#accsess the results
result = reconvert_net.results["fault8"]



{'fault1': Fault(name='fault1', description='A fault at bus 1', bus='bus1', scalings={0.0: 1.0, 50.0: 1.0, 250.0: 1.0}, active=False), 'fault2': Fault(name='fault2', description='A fault at bus 2', bus='bus2', scalings={0.0: 1.0, 50.0: 1.0, 250.0: 1.0}, active=False), 'fault3': Fault(name='fault3', description='A fault at bus 3', bus='bus3', scalings={0.0: 1.0, 50.0: 1.0, 250.0: 1.0}, active=False), 'fault4': Fault(name='fault4', description='A fault at bus 4', bus='bus4', scalings={0.0: 1.0, 50.0: 1.0, 250.0: 1.0}, active=False), 'fault5': Fault(name='fault5', description='A fault at bus 5', bus='bus5', scalings={0.0: 1.0, 50.0: 1.0, 250.0: 1.0}, active=False), 'fault6': Fault(name='fault6', description='A fault at bus 6', bus='bus6', scalings={0.0: 1.0, 50.0: 1.0, 250.0: 1.0}, active=False), 'fault7': Fault(name='fault7', description='A fault at bus 7', bus='bus7', scalings={0.0: 1.0, 50.0: 1.0, 250.0: 1.0}, active=False), 'fault8': Fault(name='fault8', description='A fault at bus 8'

In [4]:
#test the network save to database function
gi.start_dbsession()
gi.save_network_to_db(network=net, overwrite=True)


loaded_net = gi.load_network_from_db(name="MyTestNetwork")
print(f"Length of buses of the loaded network: {len(loaded_net.buses)}")
print(f"Length of buses of the original network: {len(net.buses)}")
gi.close_dbsession()

Database session started with 'grounding.db'.
Length of buses of the loaded network: 29
Length of buses of the original network: 29
Database session closed.


In [5]:
#Test the saving of the bus type and branch type
gi.start_dbsession()
#create a dummy bus type
bus_type = BusType(
    name="DummyBusType",
    description="A dummy bus type",
    system_type="Grounded",
    voltage_level=20,
    impedance_formula="rho / 100 + f / 200 + I * sqrt(f / 20)",
)
#save the bustype
gi.save_bustype_to_db(bus_type, overwrite=True)

#check the database interface for bus types
bus_types = gi.load_bustypes_from_db()

#print all names of the bus types
for name, bt in bus_types.items():
    #iterate over all properties of a bustype
    for prop in bt.__dict__:
        print(f"{prop}: {bt.__dict__[prop]}")
        #print a space
    print("\n")
    



#save the branch with the name "branch_ohl"
gi.save_branchtype_to_db(branch_ohl, overwrite=True)

#check the database interface for branch types
branch_types = gi.load_branchtypes_from_db()

#print all names of the branch types
for name, bt in branch_types.items():
    #iterate over all properties of a bustype
    for prop in bt.__dict__:
        print(f"{prop}: {bt.__dict__[prop]}")
    print("\n")
gi.close_dbsession()
    

Database session started with 'grounding.db'.
name: BusTypeFormulaTest
description: Example bus type with parameters
system_type: Grounded
voltage_level: 230.0
impedance_formula: rho * 0 + 1 + I * f * 1/50


name: DummyBusType
description: A dummy bus type
system_type: Grounded
voltage_level: 20.0
impedance_formula: rho / 100 + f / 200 + I * sqrt(f / 20)


name: TestBranchType
description: A test branch type
grounding_conductor: True
self_impedance_formula: (rho * 0 + 0.25 + j * f * 0.012)*l
mutual_impedance_formula: (rho * 0 + 0.0 + j * f * 0.010)*l


name: OHLine
description: An overhead line
grounding_conductor: False
self_impedance_formula: NaN
mutual_impedance_formula: NaN


Database session closed.
